In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather
import glob
import os
import pyarrow
from xclim import ensembles as ens
from xclim import subset

In [2]:
import matplotlib.pyplot as plt

In [3]:
folder = "/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/"

# Select variable

In [46]:
#Variable 
#var="tg_mean"
#var ="tn_mean"
#var ="tx_mean"
var ="solidprcptot"
#var ="prcptot"
#var ="dlyfrzthw"
#var ="DJC"


files85 = glob.glob(folder+"*"+"rcp85_"+var+"_monthly.nc")
files45 = glob.glob(folder+"*"+"rcp45_"+var+"_monthly.nc")
filesobs = glob.glob(folder+"NRCAN_obs_"+var+"_monthly.nc")

#ex: ACCESS1-3_rcp45_tn_mean_annual.nc
files85
files45


['/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/ACCESS1-3_rcp45_solidprcptot_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/BNU-ESM_rcp45_solidprcptot_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CanESM2_rcp45_solidprcptot_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CMCC-CMS_rcp45_solidprcptot_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/GFDL-ESM2M_rcp45_solidprcptot_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/HadGEM2-CC_rcp45_solidprcptot_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/INM-CM4_rcp45_solidprcptot_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5A-LR_rcp45_solidprcptot_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5B-LR_rcp45_solidprcptot_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/MPI-ESM-LR_rcp45_solidprcptot_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/NorESM

# Select month

In [47]:
mon = "january"
#mon = "february"
#mon = "march"
#mon = "april"
#mon = "may"
#mon = "june"
#mon = "july"
#mon = "august"
#mon = "september"
#mon = "october"
#mon = "november"
#mon = "december"

In [48]:
month_dict = {"january":1, "february":2, "march":3, "april":4, "may":5, "june":6, "july":7, 
              "august":8, "september":9, "october":10, "november":11, "december":12 }

In [49]:
month_dictinv = {1:"january", 2:"february", 3:"march", 4:"april", 5:"may", 6:"june", 7:"july", 
              8:"august", 9:"september", 10:"october", 11:"november", 12:"december"}

In [50]:
monnum = month_dict[mon]


In [51]:
filesobs

['/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/NRCAN_obs_solidprcptot_monthly.nc']

# FOR RCP45 

In [52]:
dsEns45= ens.create_ensemble(files45)
tmp145 = dsEns45.sel(time=(dsEns45.time.dt.month==monnum))

In [53]:
tmp145

<xarray.Dataset>
Dimensions:       (lat: 320, lon: 416, realization: 11, time: 151)
Coordinates:
  * lat           (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * lon           (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * time          (time) datetime64[ns] 1950-01-01 1951-01-01 ... 2100-01-01
  * realization   (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    solidprcptot  (realization, time, lat, lon) float32 dask.array<chunksize=(1, 1, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.

In [54]:
perc45 = ens.ensemble_percentiles(tmp145)
print ("Percentiles Obteined")
df45 = perc45.to_dataframe()
df452 = df45.reset_index()
df45=0
df453 = df452.loc[df452["realization"] == 0].dropna()
df452 = 0
print ("Converted to DF")
dfp452 =  df453.reset_index()
df453 =0
df45C = dfp452.copy()
dfp452=0
df45C[var+"_p10"] = round(df45C[var+"_p10"],2)
df45C[var+"_p50"] = round(df45C[var+"_p50"],2)
df45C[var+"_p90"] = round(df45C[var+"_p90"],2)
print ("Transformed in Celsiuds and rounded")
df45Cf = df45C.drop(columns=["realization"])
df45C=0

    

Percentiles Obteined
Converted to DF
Transformed in Celsiuds and rounded


In [55]:
dsEns45=0
tmp145 =0

In [56]:
#df45.reset_index().to_feather("/home/mlopez/EXEC/Processed data to clip with regions/df45"+var+"_forTS_monthly.feather")

In [57]:
#df45 = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/df45"+var+"_forTS_monthly.feather")

# FOR RCP85 

In [58]:
dsEns85= ens.create_ensemble(files85)
dsEns85.time.dt.month[0:12]
tmp185 = dsEns85.sel(time=(dsEns85.time.dt.month==monnum))

In [59]:
perc85 = ens.ensemble_percentiles(tmp185)
print ("Percentiles Obteined")
df85 = perc85.to_dataframe()
df852 = df85.reset_index()
df85=0
df853 = df852.loc[df852["realization"] == 0].dropna()
df852=0
print ("Converted to DF")
dfp852 =  df853.reset_index()
df853=0
df85C = dfp852.copy()
df85C[var+"_p10"] = round(df85C[var+"_p10"],2)
df85C[var+"_p50"] = round(df85C[var+"_p50"],2)
df85C[var+"_p90"] = round(df85C[var+"_p90"],2)
print ("Transformed in Celsiuds and rounded")
df85Cf = df85C.drop(columns=["realization"])

Percentiles Obteined
Converted to DF
Transformed in Celsiuds and rounded


In [60]:
listds85=0
dsEns85=0
tmp185

<xarray.Dataset>
Dimensions:       (lat: 320, lon: 416, realization: 11, time: 151)
Coordinates:
  * lat           (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * lon           (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * time          (time) datetime64[ns] 1950-01-01 1951-01-01 ... 2100-01-01
  * realization   (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    solidprcptot  (realization, time, lat, lon) float32 dask.array<chunksize=(1, 1, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.

In [61]:
#df85.reset_index().to_feather("/home/mlopez/EXEC/Processed data to clip with regions/df85"+var+"_forTS_monthly.feather")

In [62]:
#df85 = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/df85"+var+"_forTS_monthly.feather")

# OBSERVED DATA

In [63]:
dsOBs= xr.open_mfdataset(filesobs)

will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  dsOBs= xr.open_mfdataset(filesobs)

to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option to
open_mfdataset).
  combined = auto_combine(



In [64]:
dsOBs

<xarray.Dataset>
Dimensions:       (lat: 510, lon: 1068, time: 768)
Coordinates:
  * time          (time) datetime64[ns] 1950-01-01 1950-02-01 ... 2013-12-01
  * lon           (lon) float32 -140.95833 -140.875 ... -52.125004 -52.04167
  * lat           (lat) float32 83.45833 83.375 83.291664 ... 41.125 41.041668
Data variables:
    solidprcptot  (time, lat, lon) float32 dask.array<chunksize=(768, 510, 1068), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           NRCAN 10km Gridded Climate Dataset
    history:         2012-10-19T16:38:49: Convert from original format to NetCDF
    institution:     NRCAN
    source:          ANUSPLIN
    redistribution:  Redistribution policy unknown. For internal use only.

## Convert in Dataframe

In [65]:
dfperObs = dsOBs.to_dataframe().dropna()
dfmeanObs = dfperObs.reset_index()
dfperObs = 0
dsOBs=0

In [66]:
dfObs = dfmeanObs.rename(columns={var: var+'_Obs'})
dfmeanObs=0

In [67]:
dfObs["month"]= dfObs.time.dt.month.map(lambda m: month_dictinv[m])

In [68]:
dfObs

,lat,lon,time,solidprcptot_Obs,month
0,83.125000,-74.541672,1950-01-01,6.63,january
1,83.125000,-74.541672,1950-02-01,8.13,february
2,83.125000,-74.541672,1950-03-01,3.47,march
3,83.125000,-74.541672,1950-04-01,3.56,april
4,83.125000,-74.541672,1950-05-01,9.79,may
...,...,...,...,...,...
197577979,41.708332,-82.208336,2013-08-01,0.00,august
197577980,41.708332,-82.208336,2013-09-01,0.00,september
197577981,41.708332,-82.208336,2013-10-01,0.00,october
197577982,41.708332,-82.208336,2013-11-01,3.46,november


In [69]:
dfObs2 = dfObs.loc[dfObs['month'] == mon]
dfObs=0

In [70]:
dfObs2

,lat,lon,time,solidprcptot_Obs,month
0,83.125000,-74.541672,1950-01-01,6.630000,january
12,83.125000,-74.541672,1951-01-01,6.810000,january
24,83.125000,-74.541672,1952-01-01,4.250000,january
36,83.125000,-74.541672,1953-01-01,6.950000,january
48,83.125000,-74.541672,1954-01-01,2.250000,january
...,...,...,...,...,...
197577924,41.708332,-82.208336,2009-01-01,57.029995,january
197577936,41.708332,-82.208336,2010-01-01,18.180000,january
197577948,41.708332,-82.208336,2011-01-01,42.820000,january
197577960,41.708332,-82.208336,2012-01-01,15.829999,january


In [71]:
#dfObs.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcpObs"+var+"_forTS.feather")

In [72]:
#dfObs = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcpObs"+var+"_forTS.feather")

## Merge 2 rcp dataframes

In [73]:
df2m =  pd.merge(df45Cf, df85Cf, on=["lat","lon", "time"], suffixes=('_45', '_85'))
df45Cf = 0
df85Cf = 0

In [74]:
df2m

,index_45,lat,lon,time,solidprcptot_p10_45,solidprcptot_p50_45,solidprcptot_p90_45,index_85,solidprcptot_p10_85,solidprcptot_p50_85,solidprcptot_p90_85
0,0,66.623306,-89.045212,1950-01-01,3.76,10.17,16.33,0,3.57,10.03,16.24
1,1,66.623306,-89.045212,1951-01-01,4.82,8.28,15.35,1,4.73,8.34,15.25
2,2,66.623306,-89.045212,1952-01-01,4.40,11.07,14.83,2,4.42,11.29,14.84
3,3,66.623306,-89.045212,1953-01-01,4.91,8.64,15.27,3,4.70,8.61,15.45
4,4,66.623306,-89.045212,1954-01-01,4.06,10.69,15.27,4,3.99,10.80,15.22
...,...,...,...,...,...,...,...,...,...,...,...
11507101,220720470,40.041039,-74.045807,2096-01-01,0.18,2.84,36.86,220720470,0.00,4.86,34.89
11507102,220720471,40.041039,-74.045807,2097-01-01,1.44,12.86,26.02,220720471,0.18,1.47,38.97
11507103,220720472,40.041039,-74.045807,2098-01-01,2.56,14.55,33.32,220720472,0.00,5.62,13.30
11507104,220720473,40.041039,-74.045807,2099-01-01,0.01,4.75,19.10,220720473,0.00,0.20,3.40


In [75]:
#df2m.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/df2m"+var+"_forTS_"+mon+".feather")

In [76]:
#df2m = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/df2m"+var+"_forTS_"+mon+".feather")

# Cut by region for rcp45, 85 and Obs

## Get the lat lon regions

In [77]:
regions_folder = "/home/mlopez/EXEC/Grids-polygons-regions/"

In [78]:
files_regions = glob.glob(regions_folder+"*"+"lat_lon.feather")
files_regions

['/home/mlopez/EXEC/Grids-polygons-regions/DDE_STF_20K_REG_FOR_VUE_Slat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/DDE_STF_20K_UA_PER_VUE_Slat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/DOM_BIOlat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/REG_ECOlat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/SDOM_BIOlat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/Secteurs_Operations_Regionaleslat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/SREG_ECOlat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/territoire_guidelat_lon.feather']

In [79]:
listdfreg=[]
for region in files_regions:
    df = pd.read_feather(region)
    listdfreg.append(df)
   

## Create DF by region

In [80]:
short_dict = {"DDE_STF_20K_REG_FOR_VUE_S": ("NM_REG_FOR", "RF"), 
            "DDE_STF_20K_UA_PER_VUE_S": ("PER_NO_UA", "UA"), 
              "DOM_BIO": ("NOM", "DB"), 
              "REG_ECO": ("NOM", "RE"), 
              "SDOM_BIO": ("NOM", "SDB"), 
              "Secteurs_Operations_Regionales": ("D_GENERAL", "SOR"), 
              "SREG_ECO": ("NOM", "SRE"),  
              "territoire_guide": ("TER_GUIDE", "TG") 
              }

for region, (name, short) in short_dict.items():
    print(region, name, short)

DDE_STF_20K_REG_FOR_VUE_S NM_REG_FOR RF
DDE_STF_20K_UA_PER_VUE_S PER_NO_UA UA
DOM_BIO NOM DB
REG_ECO NOM RE
SDOM_BIO NOM SDB
Secteurs_Operations_Regionales D_GENERAL SOR
SREG_ECO NOM SRE
territoire_guide TER_GUIDE TG


In [81]:
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [82]:
d = { 'BAS-SAINT-LAURENT':"Bas-Saint-Laurent", 
     'SAGUENAY -LAC-SAINT-JEAN': "Saguenay -Lac-Saint-Jean",
     'CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES':"Capitale-Nationale-Chaudiere-Appalaches",
     'MAURICIE-CENTRE-DU-QUÉBEC':'Mauricie-Centre-du-Quebec','OUTAOUAIS':'Outaouais', 
     'ABITIBI-TEMISCAMINGUE':'Abitibi-Temiscamingue', 'COTE-NORD':'Cote-Nord',
     'NORD-DU-QUEBEC':'Nord-Du-Quebec', 'GASPESIE-ILES-DE-LA-MADELEINE':'Gaspesie-Iles-De-La-Madeleine', 
     'LANAUDIERE':'Lanaudiere','LAURENTIDES':"Laurentides", 
     "ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL":"Estrie-Monteregie-Laval-Montreal", 
     "é": "e", "É": "E", "à": "a", "è": "e", "Î": "i", "È": "E", "ô" : "o", "Ç":"C", "ç":"c",
       } 

## Cutting rcp45 and 85 by region and getting df with 4th degree polynomial

In [83]:
def polysmooth(years, y, degree):
    coef = np.polyfit(years, y, degree)
    return np.polyval(coef,years)

In [84]:
monthlist = ["january", "february", "march", "april", "may", "june", "july", "august", "september", 
             "october", "november", "december"]

In [85]:
for region, (name, short) in short_dict.items():
    print("------------------------------------------------------------------")
    print ("1/8... Cutting by region")
    print(region, name)
    dfregion = pd.read_feather("/home/mlopez/EXEC/Grids-polygons-regions/"+region+"lat_lon.feather")
    print ("2/8... Merging region with data")
    dfdatareg45 = pd.merge(dfregion, df2m, on=["lat","lon"])
    dfdataregObs = pd.merge(dfregion, dfObs2, on=["lat","lon"])
    print ("3/8... Get mean by year for every sub-region for every month")
    dfmean45 = dfdatareg45.groupby(["time", name]).mean()
    dfmean452 = dfmean45.reset_index()
    dfmean45=0
    dfmeanObs2 = dfdataregObs.groupby(["time", name]).mean() 
    dfmeanObs =0
    dfmeanObs3 = dfmeanObs2.reset_index()
    dfmeanObs2=0
    print("Merging 3 cases")
    df3m = pd.merge(dfmean452, dfmeanObs3, how='left', on= ["time", name] )
    dfmeanObs3=0
    dfmean452=0
    print ("4/8... Split by subregion")
    df3ma = df3m[['time', name, var+'_p10_45', var+'_p50_45', var+'_p90_45',
                  var+'_p10_85', var+'_p50_85', var+'_p90_85', var+'_Obs']]
    df3m=0
    if (var == "tg_mean" or var== "tn_mean" or var=="tx_mean"):
        df3ma[var+"_Obs"] = df3ma[var+"_Obs"]- 273.15
    if (var == "solidprcptot"):
        df3ma[var+"_Obs"] = df3ma[var+"_Obs"]/10
    df3ma[var+"_Obs"] = round(df3ma[var+"_Obs"], 2)
    for name2,dfsr in df3ma.groupby(name):
        print ("                   "+name2, name)
        dfsrc = dfsr.copy()
        print ("6/8... 4th Polynomial - rcp45,85")
        years = dfsrc.time.dt.year
        dfsrc["time"] = years#.values
        for p in ('_p10_45', '_p50_45', '_p90_45', '_p10_85', '_p50_85', '_p90_85'):
            if (var == "tg_mean" or var== "tn_mean" or var=="tx_mean"):
                dfsrc[var+p] = polysmooth(years,dfsrc[var+p],4) - 273.15
            if (var == "solidprcptot"):
                dfsrc[var+p] = polysmooth(years,dfsrc[var+p],4)/10
            if (var == "prcptot", "dlyfrzthw", "DJC"):
                dfsrc[var+p] = polysmooth(years,dfsrc[var+p],4)
            dfsrc[var+p] = round(dfsrc[var+p], 2) 
        print(" -----------------------8/8 Creating CSV file for sub-region " +name2)
        dfsrc.to_csv(replace_all(name2,d)+"_"+var+"_"+mon+".csv")
        
    


------------------------------------------------------------------
1/8... Cutting by region
DDE_STF_20K_REG_FOR_VUE_S NM_REG_FOR
2/8... Merging region with data
3/8... Get mean by year for every sub-region for every month
Merging 3 cases
4/8... Split by subregion
                   ABITIBI-TEMISCAMINGUE NM_REG_FOR
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region ABITIBI-TEMISCAMINGUE
                   BAS-SAINT-LAURENT NM_REG_FOR
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region BAS-SAINT-LAURENT
                   CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES NM_REG_FOR
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES
                   COTE-NORD NM_REG_FOR
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region COTE-NORD
                   ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL NM_REG_F

 -----------------------8/8 Creating CSV file for sub-region 08664
                   08665 PER_NO_UA
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region 08665
                   08666 PER_NO_UA
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region 08666
                   08751 PER_NO_UA
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region 08751
                   08762 PER_NO_UA
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region 08762
                   08763 PER_NO_UA
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region 08763
                   08764 PER_NO_UA
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region 08764
                   09351 PER_NO_UA
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-r

                   Côte de la baie des Chaleurs NOM
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region Côte de la baie des Chaleurs
                   Côte gaspésienne NOM
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region Côte gaspésienne
                   Golfe du Saint- Laurent NOM
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region Golfe du Saint- Laurent
                   Haut massif gaspésien NOM
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region Haut massif gaspésien
                   Hautes collines de Baie-Comeau-Sept-Îles NOM
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region Hautes collines de Baie-Comeau-Sept-Îles
                   Hautes collines de Charlevoix et du Saguenay NOM
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file fo

 -----------------------8/8 Creating CSV file for sub-region Collines du lac Lareau
                   Collines du lac Musquaro NOM
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region Collines du lac Musquaro
                   Collines du lac Notawassi NOM
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region Collines du lac Notawassi
                   Collines du lac Onatchiway NOM
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region Collines du lac Onatchiway
                   Collines du lac Péribonka NOM
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region Collines du lac Péribonka
                   Collines du lac Saint-Patrice NOM
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region Collines du lac Saint-Patrice
                   Collines du lac Simoncouche NOM
6/8... 4th P

 -----------------------8/8 Creating CSV file for sub-region Plaine du lac Opémisca
                   Plaine du lac Saint-Jean et du Saguenay NOM
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region Plaine du lac Saint-Jean et du Saguenay
                   Plaines et coteaux du lac Simard NOM
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region Plaines et coteaux du lac Simard
                   Îles-de-la-Madeleine NOM
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region Îles-de-la-Madeleine
------------------------------------------------------------------
1/8... Cutting by region
territoire_guide TER_GUIDE
2/8... Merging region with data
3/8... Get mean by year for every sub-region for every month
Merging 3 cases
4/8... Split by subregion
                   1a TER_GUIDE
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-regi

In [86]:
df2m =0

In [87]:
dfregion=0
dfObs2=0